In [31]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn import linear_model
import statsmodels.api as sm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
df_crsp = pd.read_csv("/Users/vlad_cherevkov/Desktop/ADV_inv_proj/crsp_final.csv")
df_rf = pd.read_csv("/Users/vlad_cherevkov/Desktop/ADV_inv_proj/risk_free_rate.csv")
df_rf.columns = ['date', 'rf']
df = pd.merge(df_crsp, df_rf, on='date', how='left')
df['RET-rf'] = df['RET'] - df['rf']
df['VOL-1m'] = df['VOL'].shift(-1)
df["date"] = df["date"].astype(str)
df['datetime'] = df['date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))

In [33]:
g = df.groupby('date')
df_group = g.mean()

In [34]:
g.head()

,PERMNO,date,TICKER,PRC,VOL,RET,SHROUT,rf,RET-rf,VOL-1m,datetime
0,10104,20060731,ORCL,14.97,7933511.0,0.033126,5238329.0,0.0040,0.029126,8166519.0,2006-07-31
1,10104,20060831,ORCL,15.66,8166519.0,0.046092,5185000.0,0.0042,0.041892,11750943.0,2006-08-31
2,10104,20060929,ORCL,17.74,11750943.0,0.132822,5195474.0,0.0041,0.128722,7834295.0,2006-09-29
3,10104,20061031,ORCL,18.47,7834295.0,0.041150,5195474.0,0.0041,0.037050,6191134.0,2006-10-31
4,10104,20061130,ORCL,19.05,6191134.0,0.031402,5178000.0,0.0042,0.027202,9402761.0,2006-11-30
5,10104,20061229,ORCL,17.14,9402761.0,-0.100262,5182191.0,0.0040,-0.104262,7852907.0,2006-12-29
6,10104,20070131,ORCL,17.16,7852907.0,0.001167,5182191.0,0.0044,-0.003233,7234361.0,2007-01-31
7,10104,20070228,ORCL,16.43,7234361.0,-0.042541,5136000.0,0.0038,-0.046341,8725634.0,2007-02-28
8,10104,20070330,ORCL,18.13,8725634.0,0.103469,5140863.0,0.0043,0.099169,5216016.0,2007-03-30
9,10104,20070430,ORCL,18.80,5216016.0,0.036955,5140863.0,0.0044,0.032555,5305336.0,2007-04-30


In [35]:
l = ['200706','200806','200906','201006','201106','201206','201306', '201406', '201506','201606']

In [36]:
count = 0
l_avg_ret_top = []
l_avg_ret_bottom = []
df_avg_ret_top = pd.DataFrame(columns=['date', 'RET-rf'])
df_avg_ret_bottom = pd.DataFrame(columns=['date', 'RET-rf'])


for i in l:
    #print("count: ", count)
    df_tmp = df.loc[df['date'].str.startswith(i)] #go over each June
    tmp = int(l[count])
    tmp_1 = tmp-1
    df_to_sort = df.loc[df['date'].str.startswith(str(tmp_1))]

    #########################
    #below is the top 20:
    ########################
    df_sorted_top = df_to_sort.sort_values(by=['VOL'], ascending=False).head(20)

    # need to limit by date
    #first get startdate
    first = str(tmp) + '01'
    data = datetime.strptime(first,'%Y%m%d')
    start_date = data
    #print(start_date)
    data_1 = data - timedelta(days=1) #subtract one day
    data_1 = data_1 + relativedelta(months=+12)

    end_date = data_1
    #print(end_date)

    #see how well they perform over year:
    mask_date = (df['datetime'] > start_date) & (df['datetime'] <= end_date)
    df_year_top = df.loc[mask_date]

    company_l_top = list(df_sorted_top['PERMNO'])
    mask_comp_top = df['PERMNO'].isin(company_l_top)
    df_year_top = df_year_top.loc[mask_comp_top] #now we have only those companies in that year

    df_avg_ret_top = df_avg_ret_top.append(df_year_top[['date','RET-rf']])

    #########################
    #below is the bottom 20:
    ########################
    df_sorted_bottom = df_to_sort.sort_values(by=['VOL'], ascending=False).tail(20)
    avg_ret_bottom = df_sorted_bottom.mean()

    df_year_bottom = df.loc[mask_date]
    company_l_bottom = list(df_sorted_bottom['PERMNO'])
    mask_comp_bottom = df['PERMNO'].isin(company_l_bottom)
    df_year_bottom = df_year_bottom.loc[mask_comp_bottom]


    df_avg_ret_bottom = df_avg_ret_bottom.append(df_year_bottom[['date','RET-rf']])




    count+=1

In [70]:
###group by date:
#top
g_top =  df_avg_ret_top.groupby('date')
df_avg_ret_top_group = g_top.mean()
df_avg_ret_top_group = df_avg_ret_top_group.dropna()
#bottom
g_bottom =  df_avg_ret_bottom.groupby('date')
df_avg_ret_bottom_group = g_bottom.mean()
df_avg_ret_bottom_group = df_avg_ret_bottom_group.dropna()
#top-bottom (long-short)
df_top_bottom = pd.merge(df_avg_ret_top_group, df_avg_ret_bottom_group, on='date', how='left')
df_top_bottom = df_top_bottom.rename(columns={'RET-rf_x': 'RET-rf_top', 'RET-rf_y': 'RET-rf_b'})

df_bottom_top = df_top_bottom

df_top_bottom['RET-rf_top_b'] = df_top_bottom['RET-rf_top'] - df_top_bottom['RET-rf_b']
# del df_top_bottom['RET-rf_top']
# del df_top_bottom['RET-rf_b']

In [74]:
df_avg_ret_bottom_group.head()

,RET-rf
date,
20070629,-0.007554
20070731,-0.011681
20070831,-0.004646
20070928,0.013737
20071031,0.016873


In [71]:
df_bottom_top['RET-rf_bottom_t'] = df_bottom_top['RET-rf_b'] - df_bottom_top['RET-rf_top']
# del df_bottom_top['RET-rf_b']
# del df_bottom_top['RET-rf_top']
df_bottom_top.head()


,RET-rf_top,RET-rf_b,RET-rf_top_b,RET-rf_bottom_t
date,,,,
20070629,-0.003003,-0.007554,0.004551,-0.004551
20070731,-0.027337,-0.011681,-0.015656,0.015656
20070831,0.002828,-0.004646,0.007474,-0.007474
20070928,0.037724,0.013737,0.023987,-0.023987
20071031,0.010969,0.016873,-0.005903,0.005903


In [72]:
df_bottom_top.to_excel(r'/Users/vlad_cherevkov/fama_bottom_top.xlsx')

In [7]:
df_avg_ret_top_group.head()

,RET-rf
date,
20070629,-0.003003
20070731,-0.027337
20070831,0.002828
20070928,0.037724
20071031,0.010969


In [8]:
df_avg_ret_bottom_group.head()

,RET-rf
date,
20070629,-0.007554
20070731,-0.011681
20070831,-0.004646
20070928,0.013737
20071031,0.016873


In [9]:
df_top_bottom.head()

,RET-rf_top_b
date,
20070629,0.004551
20070731,-0.015656
20070831,0.007474
20070928,0.023987
20071031,-0.005903


In [10]:
#merge with fama french factors:
df_fama_french = pd.read_csv("/Users/vlad_cherevkov/Desktop/ADV_inv_proj/fama_french_factors.csv")
df_fama_french['date'] = df_fama_french['dateff']
df_fama_french["date"] = df_fama_french["date"].astype(str)
#merging top:
df_fama_bottom = pd.merge(df_avg_ret_bottom_group, df_fama_french, on='date', how='left')
#merging bottom:
df_fama_top = pd.merge(df_avg_ret_top_group, df_fama_french, on='date', how='left')
#merging top_bottom:
df_fama_top_b = pd.merge(df_top_bottom, df_fama_french, on='date', how='left')
df_fama__b = pd.merge(df_top_bottom, df_fama_french, on='date', how='left')

In [11]:
df_fama_top_b

,date,RET-rf_top_b,dateff,mktrf,smb,hml
0,20070629,0.004551,20070629,-0.0196,0.0077,-0.0113
1,20070731,-0.015656,20070731,-0.0373,-0.0251,-0.0334
2,20070831,0.007474,20070831,0.0092,-0.0013,-0.0224
3,20070928,0.023987,20070928,0.0322,-0.0229,-0.0187
4,20071031,-0.005903,20071031,0.0180,0.0022,-0.0259
5,20071130,-0.050533,20071130,-0.0483,-0.0263,-0.0118
6,20071231,0.016994,20071231,-0.0087,0.0020,-0.0052
7,20080131,-0.016302,20080131,-0.0636,-0.0089,0.0365
8,20080229,-0.000223,20080229,-0.0309,-0.0023,-0.0095
9,20080331,-0.017918,20080331,-0.0093,0.0094,-0.0015


In [12]:
#write to excel:
df_fama_top.to_excel(r'/Users/vlad_cherevkov/df_fama_top_final.xlsx')
df_fama_bottom.to_excel(r'/Users/vlad_cherevkov/df_fama_bottom_final.xlsx')
df_fama_top_b.to_excel(r'/Users/vlad_cherevkov/df_fama_top_b_final.xlsx')

# 3 Factor Regressions:

In [22]:
#running regression for top portfolio:
X_fama_top = df_fama_top[['mktrf','smb','hml']]
Y_fama_top = df_fama_top['RET-rf']

X_top = sm.add_constant(X_fama_top)
est_top = sm.OLS(Y_fama_top, X_top).fit()

###########################
# regression for top:
###########################
est_top.summary()
# est_top_1.params

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RET-rf   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     237.6
Date:                Sat, 18 May 2019   Prob (F-statistic):           1.22e-46
Time:                        19:16:26   Log-Likelihood:                 249.13
No. Observations:                 109   AIC:                            -490.3
Df Residuals:                     105   BIC:                            -479.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.002      0.455      0.650      -0.004       0.006
mktrf          1.2417      0.058     21.463      0.000       1.127       1.356
smb           -0.2831      0.115     -2.461      0.015      -0.511      -0.055
hml            0.6465      0.095      6.787      0.000       0.458       0.835
==============================================================================
Omnibus:                       52.794   Durbin-Watson:                   2.312
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              222.868
Skew:                           1.594   Prob(JB):                     4.02e-49
Kurtosis:                       9.238   Cond. No.                         48.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
#running regression for bottom portfolio:
X_fama_bottom = df_fama_bottom[['mktrf','smb','hml']]
Y_fama_bottom = df_fama_bottom['RET-rf']

X_bottom = sm.add_constant(X_fama_bottom)
est_bottom = sm.OLS(Y_fama_bottom, X_bottom).fit()

###########################
# regression for bottom:
###########################
est_bottom.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RET-rf   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.814
Method:                 Least Squares   F-statistic:                     158.6
Date:                Sat, 18 May 2019   Prob (F-statistic):           7.53e-39
Time:                        19:02:50   Log-Likelihood:                 229.14
No. Observations:                 109   AIC:                            -450.3
Df Residuals:                     105   BIC:                            -439.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0018      0.003      0.625      0.534      -0.004       0.008
mktrf          1.1922      0.070     17.154      0.000       1.054       1.330
smb            0.4071      0.138      2.947      0.004       0.133       0.681
hml            0.2979      0.114      2.603      0.011       0.071       0.525
==============================================================================
Omnibus:                       67.906   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              388.678
Skew:                           2.024   Prob(JB):                     3.98e-85
Kurtosis:                      11.319   Cond. No.                         48.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
#running regression for top_bottom portfolio:
X_fama_top_b = df_fama_top_b[['mktrf','smb','hml']]
Y_fama_top_b = df_fama_top_b['RET-rf_top_b']

X_top_b = sm.add_constant(X_fama_top_b)
est_top_b = sm.OLS(Y_fama_top_b, X_top_b).fit()

###########################
# regression for bottom:
###########################
est_top_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           RET-rf_top_b   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     6.553
Date:                Sat, 18 May 2019   Prob (F-statistic):           0.000418
Time:                        19:02:50   Log-Likelihood:                 201.50
No. Observations:                 109   AIC:                            -395.0
Df Residuals:                     105   BIC:                            -384.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.004     -0.191      0.849      -0.008       0.007
mktrf          0.0494      0.090      0.552      0.582      -0.128       0.227
smb           -0.6902      0.178     -3.877      0.000      -1.043      -0.337
hml            0.3486      0.147      2.364      0.020       0.056       0.641
==============================================================================
Omnibus:                       28.665   Durbin-Watson:                   2.136
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              169.587
Skew:                          -0.576   Prob(JB):                     1.50e-37
Kurtosis:                       9.001   Cond. No.                         48.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [76]:
df_fama_bottom_t = pd.merge(df_bottom_top, df_fama_french, on='date', how='left')

In [77]:
df_fama_bottom_t.head()

,date,RET-rf_top,RET-rf_b,RET-rf_top_b,RET-rf_bottom_t,dateff,mktrf,smb,hml
0,20070629,-0.003003,-0.007554,0.004551,-0.004551,20070629,-0.0196,0.0077,-0.0113
1,20070731,-0.027337,-0.011681,-0.015656,0.015656,20070731,-0.0373,-0.0251,-0.0334
2,20070831,0.002828,-0.004646,0.007474,-0.007474,20070831,0.0092,-0.0013,-0.0224
3,20070928,0.037724,0.013737,0.023987,-0.023987,20070928,0.0322,-0.0229,-0.0187
4,20071031,0.010969,0.016873,-0.005903,0.005903,20071031,0.0180,0.0022,-0.0259


In [78]:
#running regression for top_bottom portfolio:
X_fama_bottom_t = df_fama_bottom_t[['mktrf','smb','hml']]
Y_fama_bottom_t = df_fama_bottom_t['RET-rf_bottom_t']

X_bottom_t = sm.add_constant(X_fama_bottom_t)
est_bottom_t = sm.OLS(Y_fama_bottom_t, X_bottom_t).fit()

###########################
# regression for bottom:
###########################
est_bottom_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        RET-rf_bottom_t   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     6.553
Date:                Sat, 18 May 2019   Prob (F-statistic):           0.000418
Time:                        23:41:57   Log-Likelihood:                 201.50
No. Observations:                 109   AIC:                            -395.0
Df Residuals:                     105   BIC:                            -384.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007      0.004      0.191      0.849      -0.007       0.008
mktrf         -0.0494      0.090     -0.552      0.582      -0.227       0.128
smb            0.6902      0.178      3.877      0.000       0.337       1.043
hml           -0.3486      0.147     -2.364      0.020      -0.641      -0.056
==============================================================================
Omnibus:                       28.665   Durbin-Watson:                   2.136
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              169.587
Skew:                           0.576   Prob(JB):                     1.50e-37
Kurtosis:                       9.001   Cond. No.                         48.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [75]:
df_bottom_top.head()

,RET-rf_top,RET-rf_b,RET-rf_top_b,RET-rf_bottom_t
date,,,,
20070629,-0.003003,-0.007554,0.004551,-0.004551
20070731,-0.027337,-0.011681,-0.015656,0.015656
20070831,0.002828,-0.004646,0.007474,-0.007474
20070928,0.037724,0.013737,0.023987,-0.023987
20071031,0.010969,0.016873,-0.005903,0.005903


# 1 Factor Regressions (CAPM):

In [27]:
df_fama_top.head()

,date,RET-rf,dateff,mktrf,smb,hml
0,20070629,-0.003003,20070629,-0.0196,0.0077,-0.0113
1,20070731,-0.027337,20070731,-0.0373,-0.0251,-0.0334
2,20070831,0.002828,20070831,0.0092,-0.0013,-0.0224
3,20070928,0.037724,20070928,0.0322,-0.0229,-0.0187
4,20071031,0.010969,20071031,0.0180,0.0022,-0.0259


In [25]:
#running regression for top portfolio:
X_fama_top_1 = df_fama_top['mktrf']
Y_fama_top_1 = df_fama_top['RET-rf']

X_top_1_ = sm.add_constant(X_fama_top_1)
est_top_1 = sm.OLS(Y_fama_top_1, X_top_1_).fit()

###########################
# regression for top:
###########################
est_top_1.summary()
# print(est_top_1.params)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RET-rf   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     456.0
Date:                Sat, 18 May 2019   Prob (F-statistic):           2.25e-40
Time:                        19:21:22   Log-Likelihood:                 227.76
No. Observations:                 109   AIC:                            -451.5
Df Residuals:                     107   BIC:                            -446.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.003     -0.353      0.725      -0.007       0.005
mktrf          1.3211      0.062     21.353      0.000       1.198       1.444
==============================================================================
Omnibus:                       79.186   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              615.242
Skew:                           2.322   Prob(JB):                    2.52e-134
Kurtosis:                      13.672   Cond. No.                         21.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
#running regression for bottom portfolio:
X_fama_bottom_1 = df_fama_bottom['mktrf']
Y_fama_bottom_1 = df_fama_bottom['RET-rf']

X_bottom_1_ = sm.add_constant(X_fama_bottom_1)

est_bottom_1 = sm.OLS(Y_fama_bottom_1, X_bottom_1_).fit()

###########################
# regression for top:
###########################
est_bottom_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 RET-rf   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     405.0
Date:                Sat, 18 May 2019   Prob (F-statistic):           3.65e-38
Time:                        19:25:06   Log-Likelihood:                 221.24
No. Observations:                 109   AIC:                            -438.5
Df Residuals:                     107   BIC:                            -433.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0008      0.003      0.254      0.800      -0.005       0.007
mktrf          1.3218      0.066     20.125      0.000       1.192       1.452
==============================================================================
Omnibus:                       79.877   Durbin-Watson:                   2.187
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              646.792
Skew:                           2.329   Prob(JB):                    3.56e-141
Kurtosis:                      13.987   Cond. No.                         21.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
#running regression for top_bottom portfolio:
X_fama_top_b_1 = df_fama_top_b['mktrf']
Y_fama_top_b_1 = df_fama_top_b['RET-rf_top_b']

X_top_b_1_ = sm.add_constant(X_fama_top_b_1)

est_top_b_1 = sm.OLS(Y_fama_top_b_1, X_top_b_1_).fit()

###########################
# regression for bottom:
###########################
est_top_b_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           RET-rf_top_b   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                 6.615e-05
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.994
Time:                        19:26:12   Log-Likelihood:                 192.15
No. Observations:                 109   AIC:                            -380.3
Df Residuals:                     107   BIC:                            -374.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0018      0.004     -0.449      0.654      -0.010       0.006
mktrf         -0.0007      0.086     -0.008      0.994      -0.171       0.169
==============================================================================
Omnibus:                       29.975   Durbin-Watson:                   2.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              268.572
Skew:                          -0.410   Prob(JB):                     4.79e-59
Kurtosis:                      10.646   Cond. No.                         21.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [81]:
#running regression for top_bottom portfolio:
X_fama_bottom_t_1 = df_fama_bottom_t['mktrf']
Y_fama_bottom_t_1 = df_fama_bottom_t['RET-rf_bottom_t']

X_bottom_t_1_ = sm.add_constant(X_fama_bottom_t_1)
est_bottom_t_1 = sm.OLS(Y_fama_bottom_t_1, X_bottom_t_1_).fit()

###########################
# regression for bottom:
###########################
est_bottom_t_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        RET-rf_bottom_t   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                 6.615e-05
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.994
Time:                        23:51:04   Log-Likelihood:                 192.15
No. Observations:                 109   AIC:                            -380.3
Df Residuals:                     107   BIC:                            -374.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0018      0.004      0.449      0.654      -0.006       0.010
mktrf          0.0007      0.086      0.008      0.994      -0.169       0.171
==============================================================================
Omnibus:                       29.975   Durbin-Watson:                   2.241
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              268.572
Skew:                           0.410   Prob(JB):                     4.79e-59
Kurtosis:                      10.646   Cond. No.                         21.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""